In [8]:
#imports
%matplotlib inline
from os import listdir
import os.path
import calendar
from os.path import isfile, join
import sys
import subprocess
import sys, traceback
import urllib3
import json
import numpy as np
import pandas as pd
import shutil
import nltk
import csv
import string
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
import random
import gensim
from gensim.models import LdaModel
from gensim import models, corpora, similarities
import time
import re
from nltk import FreqDist
from scipy.stats import entropy
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")

In [9]:
#Get the list of stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words("english"))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/michellepetersen/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
# first get a list of all words
def compute_top_common_words(df, column, k):
    all_words = [word for item in list(df[column]) for word in item]

    # use nltk fdist to get a frequency distribution of all words
    fdist = FreqDist(all_words)
    # number of unique words
    # choose k and visually inspect the bottom 10 words of the top k
    top_k_words = fdist.most_common(k)
    #print the distribution of words
    print("\n{} Stats: Total:{} Min Hits:{} Words:{}".format(column, len(fdist), k, top_k_words))

    # define a function only to keep words in the top k words
    top_k_words,_ = zip(*fdist.most_common(k))
    top_k_words = set(top_k_words)
    return top_k_words

def keep_top_k_words(text):
        return [word for word in text if word in top_words_across_apps_unique]
    
def keep_unique_words(text):
        return [word for word in text if word not in top_words_across_apps_unique]

freewords = ['freeIpadApplication','freeApplication']
def scrub_free(word):
    for freeword in freewords:
        if word in freeword:
            return "free"
    return word

In [49]:
#Filter description
import html.parser
def build_word_list (desc):
    
    global word_df_loc
    desc = desc.encode('ascii', 'ignore').decode('ascii')
    filtered_sentence = [w for w in desc.split(" ") if not w in stop_words]
    new_description = []
    
    for word in filtered_sentence:
        if len(word) == 0:
            continue
        word = HTMLParser.HTMLParser().unescape(word)
        word = word.replace('?', ' ').replace(')', ' ').replace('(', ' ').replace('*', '').replace('.', ' ').replace('&amp;', ' ').replace(',', ' ')
        word = word.strip().replace('-', '').replace('!', '')
        new_description.append(word)
 
    # scrubbed words for this app_id
    word_list = " ".join(str(x) for x in new_description)
    return word_list

In [50]:
# Load application category data
appcatfile = open('play_app_id_to_category.json')
play_cat_info = json.loads(appcatfile.read())

#Load ios application category data
iosappcatfile = open('ios_app_id_to_category.json')
ios_cat_info = json.loads(iosappcatfile.read())

#Load play application testai category data
playapptestaicatfile = open('play_app_id_to_testai_category.json')
play_testai_cat_info = json.loads(playapptestaicatfile.read())

#Load ios application testai category data
iosapptestaicatfile = open('ios_app_id_to_testai_category.json')
ios_testai_cat_info = json.loads(iosapptestaicatfile.read())

In [43]:
#open play store data 
with open('play_app_store.json') as json_data:
    play_store_data = json.load(json_data)
 
# Hardcoded size for now but would like a better way to do this
# Create dataframe for play store application data
play_app_df = pd.DataFrame(index=range(3500), columns=("app_id","category","subcategory","description","installs","found_position", "name","rating",\
                                                   "updated","dayofweek"))
play_df_loc = 0

for category in play_store_data.keys():
    for app in play_store_data[category]:
        if app['app_id'] in play_cat_info['categories'].keys():
            if 'description' in app.keys():
                play_app_df.iloc[play_df_loc, play_app_df.columns.get_loc("category")] = category
                if app['app_id'] in play_cat_info['subcategories'].keys():
                    subcategory = play_cat_info['subcategories'].get(app['app_id'])
                    play_app_df.iloc[play_df_loc, play_app_df.columns.get_loc("subcategory")] = subcategory
                play_app_df.iloc[play_df_loc, play_app_df.columns.get_loc("app_id")] = app['app_id']
                play_app_df.iloc[play_df_loc, play_app_df.columns.get_loc("found_position")] = app['found_position']
                play_app_df.iloc[play_df_loc, play_app_df.columns.get_loc("installs")] = int(app['installs'].replace(',',''))
                play_app_df.iloc[play_df_loc, play_app_df.columns.get_loc("name")] = app['name']
                play_app_df.iloc[play_df_loc, play_app_df.columns.get_loc("rating")] = float(app['rating'])
                update_time = pd.to_datetime(app['updated'])
                play_app_df.iloc[play_df_loc, play_app_df.columns.get_loc("updated")] = update_time
                play_app_df.iloc[play_df_loc, play_app_df.columns.get_loc("dayofweek")] = calendar.day_name[update_time.weekday()]
                word_list = build_word_list(app['description'])
                play_app_df.iloc[play_df_loc, play_app_df.columns.get_loc("description")] = word_list
                play_df_loc += 1
                
# Drop null rows
play_app_df.dropna(axis = 0, inplace=True) 
#Clean out categories that we want to blacklist
play_app_df = play_app_df[play_app_df['category'] != "librariesanddemo"]
play_app_df.tail()

JSONDecodeError: Expecting value: line 928 column 25 (char 172370)

In [32]:
#open ios store data 
with open('ios_app_store.json') as json_data:
    ios_store_data = json.load(json_data)
 
# Hardcoded size for now but would like a better way to do this
# Create dataframe for play store application data
ios_app_df = pd.DataFrame(index=range(20000), columns=("ios_app_id","ios_category","ios_found_position","ios_name","ios_num_ratings","ios_rating",\
                                                       "ios_origin_list","ios_store_url","ios_version","ios_website"))
ios_df_loc = 0

for app in ios_store_data:
    ios_app_df.iloc[ios_df_loc, ios_app_df.columns.get_loc("ios_app_id")] = app['app_id']
    ios_app_df.iloc[ios_df_loc, ios_app_df.columns.get_loc("ios_category")] = app['category']
    ios_app_df.iloc[ios_df_loc, ios_app_df.columns.get_loc("ios_found_position")] = app['found_position']
    ios_app_df.iloc[ios_df_loc, ios_app_df.columns.get_loc("ios_name")] = app['name']
    ios_app_df.iloc[ios_df_loc, ios_app_df.columns.get_loc("ios_num_ratings")] = float(app['num_ratings'])
    ios_app_df.iloc[ios_df_loc, ios_app_df.columns.get_loc("ios_rating")] = float(app['rating'])
    ios_app_df.iloc[ios_df_loc, ios_app_df.columns.get_loc("ios_origin_list")] = app['origin_list']
    ios_app_df.iloc[ios_df_loc, ios_app_df.columns.get_loc("ios_store_url")] = app['store_url']
    ios_app_df.iloc[ios_df_loc, ios_app_df.columns.get_loc("ios_version")] = app['version']  
    ios_app_df.iloc[ios_df_loc, ios_app_df.columns.get_loc("ios_website")] = app['website']  
    ios_df_loc += 1
                
# Drop null rows
ios_app_df.dropna(axis = 0, inplace=True)  
ios_app_df.tail()

KeyboardInterrupt: 

In [48]:
ios_app_df.head(50)

,ios_app_id,ios_category,ios_found_position,ios_name,ios_num_ratings,ios_rating,ios_origin_list,ios_store_url,ios_version,ios_website
0,349554263,Reference,0,Ancestry,10400,4.6,appsByRevenue,https://itunes.apple.com/us/app/id349554263,9.9,http://www.ancestry.com/ancestry-app
1,1385387689,Reference,2,Daily Prophesy,241,1.4,appsByRevenue,https://itunes.apple.com/us/app/id1385387689,2018-07-07,http://www.apple.com/itunes/download/
2,1273077875,Reference,3,Horoscope - Daily Horoscope,20700,4.4,appsByRevenue,https://itunes.apple.com/us/app/id1273077875,1.4,http://www.apple.com/itunes/download/
3,1255066618,Reference,5,30 Day Bible Study,8600,4.8,appsByRevenue,https://itunes.apple.com/us/app/id1255066618,1.5.2,http://www.apple.com/itunes/download/
4,1368379415,Reference,6,Big Brain Quiz,273,1,appsByRevenue,https://itunes.apple.com/us/app/id1368379415,2018-07-07,http://www.apple.com/itunes/download/
5,1386636398,Reference,7,Wood Magic,70,1.3,appsByRevenue,https://itunes.apple.com/us/app/id1386636398,2018-07-07,http://www.apple.com/itunes/download/
6,287734809,Reference,8,Whitepages People Search,33900,4.6,appsByRevenue,https://itunes.apple.com/us/app/id287734809,3.3.4,http://schema.org
7,1390916375,Reference,10,Soul Reader 2018,23,1.2,appsByRevenue,https://itunes.apple.com/us/app/id1390916375,2018-07-07,http://www.apple.com/itunes/download/
8,1348028646,Reference,11,Translate Now - Translator,6900,4.6,appsByRevenue,https://itunes.apple.com/us/app/id1348028646,1.1.2,http://wzp.solutions/translate-now
9,576588894,Reference,12,Sky Guide,74900,4.9,appsByRevenue,https://itunes.apple.com/us/app/id576588894,6.7.1,http://fifthstarlabs.com


In [189]:
print("Apps by Revenue")
Most_popular_rev_apps = ios_app_df.loc[ios_app_df['ios_origin_list'] == 'appsByRevenue']
print(len(Most_popular_rev_apps))
Most_popular_rev_apps = Most_popular_rev_apps.drop_duplicates(subset=['ios_app_id'])
print(len(Most_popular_rev_apps))
Most_popular_rev_apps['ios_num_ratings'].describe()
Most_popular_rev_apps = Most_popular_rev_apps.loc[Most_popular_rev_apps['ios_num_ratings'] > float(300.0)]
print(len(Most_popular_rev_apps))
Most_popular_rev_apps = Most_popular_rev_apps.loc[Most_popular_rev_apps['ios_rating'] >= float(3.0)]
print(len(Most_popular_rev_apps))
Most_popular_rev_apps = Most_popular_rev_apps.loc[Most_popular_rev_apps['ios_found_position'] <= 25]
print(len(Most_popular_rev_apps))
out = Most_popular_rev_apps.to_json(orient='records', lines=True)
with open('most_popular_revenue_apps.json', 'w') as f:
    f.write(out)  
    
print("Apps Paid")
Most_popular_paid_apps = ios_app_df.loc[ios_app_df['ios_origin_list'] == 'paidApplications']
print(len(Most_popular_paid_apps))
Most_popular_paid_apps = Most_popular_paid_apps.drop_duplicates(subset=['ios_app_id'])
print(len(Most_popular_paid_apps))
Most_popular_paid_apps = Most_popular_paid_apps.loc[Most_popular_paid_apps['ios_num_ratings'] >= float(300.0)]
print(len(Most_popular_paid_apps))
Most_popular_paid_apps = Most_popular_paid_apps.loc[Most_popular_paid_apps['ios_rating'] >= float(3.0)]
print(len(Most_popular_paid_apps))
Most_popular_paid_apps = Most_popular_paid_apps.loc[Most_popular_paid_apps['ios_found_position'] <= 25]
print(len(Most_popular_paid_apps))
out = Most_popular_paid_apps.to_json(orient='records', lines=True)
with open('most_popular_paid_apps.json', 'w') as f:
    f.write(out) 
    
print("Apps Free")
most_popular_free_apps = ios_app_df.loc[ios_app_df['ios_origin_list'] == 'freeApplications']
print(len(most_popular_free_apps))
most_popular_free_apps = most_popular_free_apps.drop_duplicates(subset=['ios_app_id'])
print(len(most_popular_free_apps))
most_popular_free_apps = most_popular_free_apps.loc[most_popular_free_apps['ios_num_ratings'] >= float(300.0)]
print(len(most_popular_free_apps))
most_popular_free_apps = most_popular_free_apps.loc[most_popular_free_apps['ios_rating'] >= float(3.0)]
print(len(most_popular_free_apps))
most_popular_free_apps = most_popular_free_apps.loc[most_popular_free_apps['ios_found_position'] <= 50]
print(len(most_popular_free_apps))
out = most_popular_free_apps.to_json(orient='records', lines=True)
with open('most_popular_free_apps.json', 'w') as f:
    f.write(out)  
    

# Display most frequent category
category_df = ios_app_df.groupby(['ios_category']).size().nlargest(50).reset_index(name='App Count')
print('Top Categories by Application Count: ', category_df.head(50))

# Display most ratings (maybe this gives #installs)
category_df = ios_app_df.pivot_table(index='ios_category', values="ios_num_ratings", aggfunc='sum')
category_df.sort_values(by=['ios_num_ratings', 'ios_category'], inplace=True, ascending=False)
print('Top Categories by Number of Ratings: ', category_df.head(50))

# Display most frequent combination of category and origin_list
most_popular_combo = ios_app_df.groupby(['ios_category','ios_origin_list']).size().nlargest(150).reset_index(name='App Count')
most_popular_combo.sort_values(by=['ios_category', 'ios_origin_list', 'App Count'], inplace=True, ascending=True)
print('Top category and origin_list by app count: ', most_popular_combo.head(150))

#def contains(text)
#   return [word for word in text if word not in top_words_across_apps_unique]
#most_popular_ipad = most_popular_apps.loc[most_popular_apps['ios_name'].contains('ipad')]
#len(most_popular_ipad)
#most_popular_apps = most_popular_apps.groupby(['category','ios_found_position']).size().nlargest(2000).reset_index(name='App Count')

Apps by Revenue
4182
4026
2180
2144
435
Apps Paid
3186
3141
739
735
282
Ipad Paid
2982
2950
614
282
217
Merged Paid
1233
1154
Apps Free
4672
4578
3216
3063
978
Apps Free Ipad
4475
4376
2725
2627
904
Merged Free
1882
1337
Merged All
3332
2232
2056
2056
('Top category by app count: ',          ios_category  App Count
0               Games        126
1       Photo & Video        115
2             Finance        109
3    Health & Fitness        109
4           Utilities        109
5            Stickers        104
6        Productivity        104
7       Entertainment        102
8           Lifestyle        100
9            Business         92
10             Travel         90
11          Education         88
12              Music         87
13  Social Networking         86
14           Shopping         85
15       Food & Drink         77
16            Medical         74
17          Reference         69
18               News         69
19            Weather         68
20              Books  

/Users/michellepetersen/anaconda3/lib/python2.7/site-packages/ipykernel/__main__.py:48: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/Users/michellepetersen/anaconda3/lib/python2.7/site-packages/ipykernel/__main__.py:96: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [177]:
category_origin_df = ios_app_df.pivot_table(index=['ios_category', 'ios_origin_list'], values='ios_num_ratings', aggfunc='sum')
category_origin_df.sort_values(by=['ios_num_ratings', 'ios_origin_list','ios_category'], inplace=True, ascending=False)
print('Top category and origin_list by Number ratings: ')
category_origin_df.reset_index().to_json('ratings.json',orient='records',lines=True)
category_origin_df.reset_index().to_csv('ratings.csv')

Top category and origin_list by Number ratings: 


In [62]:
#Analyze subcategories

sub_df_loc = 0
sub_df = ios_app_df.loc[ios_app_df['ios_category'] == 'Productivity']
sub_df = sub_df.loc[sub_df['ios_origin_list'] != "freeIpadApplications"]
sub_df = sub_df.loc[sub_df['ios_origin_list'] != "paidIpadApplications"]
sub_df = sub_df.drop_duplicates(subset=['ios_app_id'])
sub_df = sub_df.loc[sub_df['ios_found_position'] <= 50]
    
len(sub_df)
print('Top Apps: ', sub_df.head(300))
sub_df.reset_index().to_json('productivity_apps.json',orient='records',lines=True)

TypeError: string indices must be integers

In [ ]:
# Display most frequent category
category_df = play_app_df.groupby(['category']).size().nlargest(300).reset_index(name='App Count')
print('Top Categories by Application Count: ', category_df)

# Display most installs
category_df = play_app_df.pivot_table(index='category', values='installs', aggfunc='sum')
category_df.sort_values(by=['installs', 'category'], inplace=True, ascending=False)
print('Top Categories by Number Installs: ', category_df.head(300))

# Display most frequent subcategory
most_popular_combo = play_app_df.groupby(['subcategory']).size().nlargest(300).reset_index(name='App Count')
print('Top Subcategories by Application Count: ', most_popular_combo)

# Display most installs
subcategory_df = play_app_df.pivot_table(index='subcategory', values='installs', aggfunc='sum')
subcategory_df.sort_values(by=['installs', 'subcategory'], inplace=True, ascending=False)
print('Top Subcategories by Number Installs: ', subcategory_df.head(300))

In [ ]:
# Display most frequent combination of category and subcategory
most_popular_combo = play_app_df.groupby(['category', 'subcategory']).size().nlargest(400).reset_index(name='App Count')
print('Top Fifty Most Popular category and subcategory by app count: ', most_popular_combo)

# Display most installs
category_df = play_app_df.pivot_table(index=['category', 'subcategory'], values='installs', aggfunc='sum')
category_df.sort_values(by=['installs', 'category', 'subcategory'], inplace=True, ascending=False)
print('Top Categories by Number Installs: ', category_df.head(400))
category_df.reset_index().to_json('play_category.json',orient='records',lines=True)
category_df.reset_index().to_csv('play_category.csv')

In [46]:
play_app_df['dayofweek'].value_counts()
#Most popular days to update an app are Monday and Tuesday

Monday       633
Tuesday      620
Thursday     583
Friday       538
Wednesday    506
Sunday       181
Saturday     175
Name: dayofweek, dtype: int64

Coefficient of Correlation
Value of r	Strength of relationship
-1.0 to -0.5 or 1.0 to 0.5	Strong
-0.5 to -0.3 or 0.3 to 0.5	Moderate
-0.3 to -0.1 or 0.1 to 0.3	Weak
-0.1 to 0.1	None or very weak

In [ ]:
#Analyze subcategories
sub_df = play_app_df.loc[play_app_df['subcategory'] == "chat"]
print("Starting Length: {}".format(len(sub_df)))

# Calculate correlation between columns
import scipy.stats
print(scipy.stats.pearsonr(sub_df['installs'].values,sub_df['found_position'].values)[0])
print(scipy.stats.pearsonr(sub_df['rating'].values,sub_df['found_position'].values)[0])
#weak correlation
print(scipy.stats.pearsonr(sub_df['rating'].values,sub_df['installs'].values)[0])
print(scipy.stats.spearmanr(sub_df['installs'].values,sub_df['found_position'].values)[0])
print(scipy.stats.spearmanr(sub_df['rating'].values,sub_df['found_position'].values)[0])
print(scipy.stats.spearmanr(sub_df['rating'].values,sub_df['installs'].values)[0])
print(scipy.stats.kendalltau(sub_df['installs'].values,sub_df['found_position'].values)[0])
print(scipy.stats.kendalltau(sub_df['rating'].values,sub_df['found_position'].values)[0])
print(scipy.stats.kendalltau(sub_df['rating'].values,sub_df['installs'].values)[0])